<a href="https://colab.research.google.com/github/AppimateSA/AutoVisual/blob/text2video_stablediffusion/notebooks/Text-To-Video-Finetuning.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare the enviornment

In [1]:
import os
import sys
import platform
import requests
import torch

try:
    from google.colab import drive
    IN_COLAB = True
except:
    WORKING_DIR = '.'
    IN_COLAB = False
if IN_COLAB:
    WORKING_DIR = '/content'
    drive.mount('/content/drive',  force_remount=True) # Mount drive in order access Google drive
if IN_COLAB:
    sys.path.insert(0, WORKING_DIR)
else:
    # The actual code is one level higher in folder depth/structure, so we're elevating this notebook.
    sys.path.insert(0,f"{WORKING_DIR}/")

# Install

### Install Main Repo

In [2]:
# # %cd {WORKING_DIR}/drive/MyDrive/Colab Notebooks/Supervised Learning/T2V/Text-To-Video-Finetuning/
# %cd WORKING_DIR

dataFolder = os.path.expanduser('~/Desktop/AutoVisual')
# %cd {dataFolder}

### Install Main Model

In [16]:
!git lfs install
!git clone https://huggingface.co/damo-vilab/text-to-video-ms-1.7b ./models/model_scope_diffusers/
# # %cd ..

Updated Git hooks.
Git LFS initialized.
Cloning into './models/model_scope_diffusers'...
remote: Enumerating objects: 79, done.
remote: Total 79 (delta 0), reused 0 (delta 0), pack-reused 79
Unpacking objects: 100% (79/79), 523.71 KiB | 2.34 MiB/s, done.
Filtering content: 100% (12/12), 12.51 GiB | 2.37 MiB/s, done.


## Preprocess Videos
See arguments you can pass [here](https://github.com/ExponentialML/Video-BLIP2-Preprocessor#default-arguments)

In [17]:
if IN_COLAB:
    dataFolder = f'{WORKING_DIR}/drive/MyDrive/Colab Notebooks/datasets/Appimate'
else:
    dataFolder = os.path.expanduser( '~/.cache/datasets/Appimate' )
    
# print(dataFolder)
# !pwd
# # %cd {WORKING_DIR}/AutoVisual
# !python preprocess.py \
#     --input_dir '{dataFolder}' \
#     --json_file_path '{dataFolder}/dataset.json' \
#     --skip_interval 1 \
#     --output_dir '{dataFolder}/T2V'

## Train Video

In [5]:
# %%writefile ./configs/my_config_hq.yaml

# pretrained_model_path: "/content/text-to-video-ms-1.7b" #https://huggingface.co/damo-vilab/text-to-video-ms-1.7b/tree/main
# output_dir: "/content/drive/MyDrive/Colab Notebooks/Supervised Learning/T2V/AutoVisual/outputs"
# train_text_encoder: False
# #resume_from_checkpoint: None
# dataset_types:
#   - 'folder'

# train_data:
#   path: "/content/train_all_mp4"
#   preprocessed: True
#   n_sample_frames: 2
#   shuffle_frames: False
#   width: 128 #384
#   height: 128 #256
#   sample_start_idx: 0
#   sample_frame_rate: 24
#   vid_data_key: "video_path"

#   # single_video_path: ""
#   single_video_prompt: ""

# validation_data:
#   prompt: ""
#   sample_preview: True
#   num_frames: 16
#   width: 128 #384
#   height: 128 #256
#   num_inference_steps: 50
#   guidance_scale: 9

# learning_rate: 1e-5
# adam_weight_decay: 1e-2
# train_batch_size: 1
# max_train_steps: 50000
# checkpointing_steps: 2500
# validation_steps: 2500
# trainable_modules:
#   - "attentions"
# seed: 64
# mixed_precision: "fp16"
# use_8bit_adam: False # This seems to be incompatible at the moment.

# gradient_checkpointing: True
# text_encoder_gradient_checkpointing: True
# # Xformers must be installed
# enable_xformers_memory_efficient_attention: False

In [21]:
!python train.py --config ./configs/my_config_hq.yaml --experiment_num=0 --use_wandb=False

/home/luthando/miniconda3/envs/vgen/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_cuda.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.2.0+cu121 with CUDA 1201 (you have 2.2.0)
    Python  3.8.18 (you have 3.8.18)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
Initializing the conversion map
03/27/2024 15:37:30 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process

# Test Inference

In [7]:
# !python inference.py \
#     --model '/content/drive/MyDrive/Colab Notebooks/Supervised Learning/T2V/Text-To-Video-Finetuning/outputs/train_2024-01-13T20-29-09/checkpoint-50" --prompt "Simplify (x+1)(x-1) explained' \
#     --prompt 'Please simplify this expression: (x+1)(x-1)' \
#     --neg_prompt "watermark+++, text, shutterstock text, shutterstock++, blurry, ugly, username, url, low resolution, low quality" \
#     --num-frames 16 \
#     --window-size 12 \
#     --width 128 \
#     --height 128 \
#     --sdp